In [1]:
from module.engine.tensor import Tensor
from module.model.layers import Layer , Sequential
from module.model.loss import Loss
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

## Solving a regression problem with the sklearn diabetes dataset

In [2]:
#load data
X , y = datasets.load_diabetes(return_X_y=True, as_frame=False, scaled=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [3]:
model = Sequential([Layer(size=(10,30) ,activation='relu'),
                    Layer(size=(30,50),activation='tanh'),
                    Layer(size=(50,50) ,activation='tanh'),
                    Layer(size=(50,30) ,activation='tanh'),
                    Layer(size=(30,1))])
model.summary()

Layer(type)         Shape              Parameters
---------------------------------------------------
ReLULayer       Shape: (10, 30))      Params: 330
---------------------------------------------------
TanhLayer       Shape: (30, 50))      Params: 1550
---------------------------------------------------
TanhLayer       Shape: (50, 50))      Params: 2550
---------------------------------------------------
TanhLayer       Shape: (50, 30))      Params: 1530
---------------------------------------------------
LinearLayer       Shape: (30, 1))      Params: 31
---------------------------------------------------
Trainable Parameters                          5991


In [4]:
train_loss = []
test_loss = []

In [ ]:
lr = 0.1
bactch_size = 80
for epochs in range(5):
    losstti = 0
    split = 'train'
    Xtr, Ytr = {'train': (X_train, y_train), 'test': (X_test, y_test)}[split]
    for i in range(0,round(len(Xtr)/bactch_size)):
        size = np.random.randint(0, len(Xtr), bactch_size)
        train_pred = [model(Xtr[x]) for x in size]
        #losstr= Loss.mean_squared_error(Ytr[size],train_pred)
        losstr = sum((ypred - ytrue)**2 for ypred, ytrue in zip(train_pred,Ytr[size]))/len(size)
        losstti +=losstr
        # zero grad
        for p in model.parameters():
            p.grad = 0.0
        losstr.backward()
        # update
        for p in model.parameters():
            p.data =  p.data - (0.001 * p.grad)
    train_loss.append(losstti.data/round(len(Xtr)/bactch_size))
    split = 'test'
    Xtt, Ytt = {'train': (X_train, y_train), 'test': (X_test, y_test)}[split]        
    test_pred = [model(x) for x in Xtt]
    # losstt = Loss.mean_squared_error(Ytt,test_pred)
    losstt = sum((ypred - ytrue)**2 for ypred, ytrue in zip(test_pred,Ytt))/len(Ytt)
    test_loss.append(losstt.data)
    print(f'Epoch {epochs} ----------> train loss: {losstr.data} test loss: {losstt.data}')

In [ ]:
plt.plot(range(5), train_loss, label='train loss')
plt.plot(range(5), test_loss, label='test loss')
plt.legend()
plt.show()

In [285]:
xs = [
  [2.49, 3.5, -4.0],
  [3.0, -11.7, 3.5],
  [0.5, 13.0, -2.7],
  [12.37, 3.0, -1.0],
  [-9.2, 7.3, -1.0],
  [2.56, 13.0, -5.0],
  [8.0, -5.0, 21.11],
]
ys = [15, 13, 11.9, 14,35, 32, 38]
n = Sequential([Layer(size=(7,14), activation='tanh'),Layer(size=(14,10) ,activation='tanh'),Layer(size=(10,1))])
n.summary()

Layer(type)         Shape              Parameters
---------------------------------------------------
TanhLayer       Shape: (7, 14))      Params: 112
---------------------------------------------------
TanhLayer       Shape: (14, 10))      Params: 150
---------------------------------------------------
LinearLayer       Shape: (10, 1))      Params: 11
---------------------------------------------------
Trainable Parameters                          273


In [286]:
for k in range(4000): 
    p = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, p))/len(ys)
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data =  p.data - (0.001 * p.grad)
    print(k, loss.data)

0 649.8814663768356
1 572.9490721533746
2 521.3329575831168
3 472.2732754037245
4 426.02517103874453
5 380.64224560339267
6 336.89180028858823
7 295.351086905508
8 260.15639482008925
9 233.0742310328157
10 207.6225628249387
11 183.0112391559822
12 163.30807970802482
13 150.34946235049802
14 142.32113008252222
15 127.11878680789204
16 120.61228111309597
17 113.59257064927117
18 108.76501481352848
19 105.21174319488054
20 100.68045827935612
21 92.92421808264365
22 91.11651282148325
23 87.43051265321401
24 85.82277672692233
25 83.86528060181337
26 82.71659842453518
27 81.42649001725987
28 80.7380229030353
29 81.34714363666514
30 75.69180293174684
31 73.12666252559258
32 71.69872920988546
33 71.61146705332838
34 69.23679732023504
35 69.91501023299605
36 66.96018639939255
37 68.87225062959138
38 64.72016895420298
39 68.77837305428335
40 62.42927371027237
41 69.92250230110041
42 60.250635123569914
43 70.72604514736607
44 58.286472064098994
45 67.81654172309558
46 56.995309431751544
47 63.196